In [33]:
import os
import re
import httpx
from dotenv import load_dotenv
from langchain_groq import ChatGroq

In [34]:
# Load environment variables once
load_dotenv()

True

In [35]:
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
GROQ_API_KEY=os.getenv("GROQ_API_KEY")

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["GROQ_API_KEY"]= GROQ_API_KEY

In [36]:
llm=ChatGroq(model_name="llama-3.3-70b-versatile")

In [37]:
class Chatbot:
    def __init__(self,system=""):
        self.system=system
        self.message=[]
        if self.system:
            self.message.append({"role":"system","content":system})
    def __call__(self,message):
        self.message.append({"role":"user","content":message})
        result=self.execute()
        self.message.append({"role":"assistant","content":result})
        return result
        
    def execute(self):
        llm = ChatGroq(model_name="Gemma2-9b-It")
        result = llm.invoke(self.message)
        return result.content
        

In [38]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop your output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.


Your available actions are:
calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point
syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

simon_blog_search:
e.g. simon_blog_search: Python
Search Simon's blog for that term

Example session:
Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:
Observation: France is a country. The capital is Paris.

You then output:
Answer: The capital of France is Paris

Please Note: if you get basic conversation questions like "hi","hello","how are you?",\n
you have to answer "hi","hello","i am good".
""".strip()

In [39]:
import httpx
def wikipedia(query):
    response = httpx.get("https://en.wikipedia.org/w/api.php", params={
        "action": "query",
        "list": "search",
        "srsearch": query,
        "format": "json"
    })
    return response.json()["query"]["search"][0]["snippet"]

In [40]:
import httpx
def simon_blog_search(query):
    response = httpx.get("https://datasette.simonwillison.net/simonwillisonblog.json", params={
        "sql": """
        select
          blog_entry.title || ': ' || substr(html_strip_tags(blog_entry.body), 0, 1000) as text,
          blog_entry.created
        from
          blog_entry join blog_entry_fts on blog_entry.rowid = blog_entry_fts.rowid
        where
          blog_entry_fts match escape_fts(:q)
        order by
          blog_entry_fts.rank
        limit
          1
        """.strip(),
        "_shape": "array",
        "q": query,
    })
    return response.json()[0]["text"]
def calculate(number):
    return eval(number)

In [41]:
def calculate(number):
    return eval(number)

In [42]:
known_actions = {
    "wikipedia": wikipedia,
    "calculate": calculate,
    "simon_blog_search": simon_blog_search
}

In [43]:
import re
action_re = re.compile('^Action: (\w+): (.*)')

In [44]:
bot = Chatbot(prompt)

In [45]:
bot("hi how are you?")

"Understood. I'm ready for my first question.  \n"

In [46]:

def query(question,max_turns=5):
    i = 0
    bot = Chatbot(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception(f"Unknown action: {action}: {action_input}")
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = f"Observation: {observation}"
        else:
            return result
    

In [47]:

query("Fifteen * twenty five")

Thought: I need to calculate 15 * 25. 

Action: calculate: 15 * 25
PAUSE 

 -- running calculate 15 * 25
Observation: 375
Answer: 15 * 25 = 375 



'Answer: 15 * 25 = 375 \n'

In [ ]:
query("what is AI") 

Thought: I need to find a good definition of AI. Wikipedia might be a good place to start.  
Action: wikipedia: AI 
PAUSE  

 -- running wikipedia AI 
Observation: Artificial intelligence (<span class="searchmatch">AI</span>), in its broadest sense, is intelligence exhibited by machines, particularly computer systems. It is a field of research in
Thought: That's a good start. I should get more context. 

Action: wikipedia: AI 
PAUSE  

 -- running wikipedia AI 
Observation: Artificial intelligence (<span class="searchmatch">AI</span>), in its broadest sense, is intelligence exhibited by machines, particularly computer systems. It is a field of research in
Thought: I need to read more to fully understand.  

Action: wikipedia: AI 
PAUSE 

 -- running wikipedia AI 


In [ ]:

next_prompt="Tell me about the current gdp of PAKISATN  from Wikipedia."

In [ ]:
query("Has Simon written about AI?")

Thought: I need to check Simon's blog for information about AI. 

Action: simon_blog_search: AI
PAUSE  

 -- running simon_blog_search AI
Observation: It's OK to call it Artificial Intelligence: Update 9th January 2024: This post was clumsily written and failed to make the point I wanted it to make. I've published a follow-up, What I should have said about the term Artificial Intelligence which you should read instead.
My original post follows.


We need to be having high quality conversations about AI: what it can and can't do, its many risks and pitfalls and how to integrate it into society in the most beneficial ways possible.
Any time I write anything that mentions AI it's inevitable that someone will object to the very usage of the term.
Strawman: "Don't call it AI! It's not actually intelligent - it's just spicy autocomplete."
That strawman is right: it's not "intelligent" in the same way that humans are. And "spicy autocomplete" is actually a pretty good analogy for how a lot of

'Answer: Yes, Simon has written about AI. \n'

In [ ]:
print("'Answer: Yes, Simon has written about AI. \n'")

'Answer: Yes, Simon has written about AI. 
'


In [ ]:
query("can you tell me capital of usa?")

Thought: I should look up the capital of the USA on Wikipedia. 

Action: wikipedia: Washington, D.C.

PAUSE 

 -- running wikipedia Washington, D.C.
Observation: <span class="searchmatch">Washington</span>, <span class="searchmatch">D.C</span>., formally the District of Columbia and commonly known as <span class="searchmatch">Washington</span> or <span class="searchmatch">D.C</span>., is the capital city and federal district of the United States
Answer: The capital of the USA is Washington, D.C. 



'Answer: The capital of the USA is Washington, D.C. \n'